## Setup

In [ ]:
PROJECT_DIR = "/content/federated-triplet-loss"
SRC = "https://github.com/sdll/federated-triplet-loss"

GDRIVE_MOUNT_POINT = "/content/drive"
WORKSPACE= "sdll"
SCRIPT_FILENAME="federated_triplet_loss/experiments/emnist_embeddings/train.py"

In [ ]:
from google.colab import drive
import os
drive.mount(GDRIVE_MOUNT_POINT)
GOOGLE_DRIVE_ROOT = GDRIVE_MOUNT_POINT + "/" + list(filter(lambda x: x[0] != '.', os.listdir(GDRIVE_MOUNT_POINT)))[0]
GOOGLE_DRIVE_TARGET_DIR = "{}/federated-triplet-loss".format(GOOGLE_DRIVE_ROOT)

Mounted at /content/drive


In [ ]:
![ -d $PROJECT_DIR ] || git clone $SRC $PROJECT_DIR

In [ ]:
%cd $PROJECT_DIR

/content/federated-triplet-loss


In [ ]:
!pip install -q --force tensorflow-addons
!pip install -q --force tensorflow-privacy

ERROR: tensorflow-federated 0.14.0 has requirement tensorflow-addons~=0.9.1, but you'll have tensorflow-addons 0.10.0 which is incompatible.
ERROR: tensorflow-federated 0.14.0 has requirement tensorflow-addons~=0.9.1, but you'll have tensorflow-addons 0.10.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install -q -r requirements.txt

## Utilities

In [ ]:
import signal
import subprocess as sp
import sys
import fileinput
from glob import glob
from pathlib import Path
import os

class VerboseCalledProcessError(sp.CalledProcessError):
    def __str__(self):
        if self.returncode and self.returncode < 0:
            try:
                msg = "Command '%s' died with %r." % (
                    self.cmd,
                    signal.Signals(-self.returncode),
                )
            except ValueError:
                msg = "Command '%s' died with unknown signal %d." % (
                    self.cmd,
                    -self.returncode,
                )
        else:
            msg = "Command '%s' returned non-zero exit status %d." % (
                self.cmd,
                self.returncode,
            )

        return f"{msg}\n" f"Stdout:\n" f"{self.output}\n" f"Stderr:\n" f"{self.stderr}"


def bash(cmd, print_stdout=True, print_stderr=True):
    proc = sp.Popen(
        cmd,
        stderr=sp.PIPE,
        stdout=sp.PIPE,
        shell=True,
        universal_newlines=True,
        executable="/bin/bash",
    )

    all_stdout = []
    all_stderr = []
    while proc.poll() is None:
        for stdout_line in proc.stdout:
            if stdout_line != "":
                if print_stdout:
                    print(stdout_line, end="")
                all_stdout.append(stdout_line)
        for stderr_line in proc.stderr:
            if stderr_line != "":
                if print_stderr:
                    print(stderr_line, end="", file=sys.stderr)
                all_stderr.append(stderr_line)

    stdout_text = "".join(all_stdout)
    stderr_text = "".join(all_stderr)
    if proc.wait() != 0:
        raise VerboseCalledProcessError(proc.returncode, cmd, stdout_text, stderr_text)

def stringify_options(options):
    return  [
        "--{} {}".format(key, "" if isinstance(value, bool) else value)
        for key, value in options.items()
    ]


def filename_to_module(filename):
    return filename.replace("/", ".")[:-3]


## Experiments

In [ ]:
import numpy as np

N_EXPERIMENTS = 1

options_grid = {
    "dp_l2_norm_clip": (1e-2, 0.5),
    "dp_noise_multiplier": (1e-3, .1)
} 

options = {
    "num_rounds": 1000,
    "num_clients_per_round": 10,
    "num_client_disc_train_steps": 6,
    "num_server_gen_train_steps": 6,
    "num_rounds_per_eval": 10,
    "num_rounds_per_save_images": 10,
    "use_dp": True,
    "dp_l2_norm_clip": 0.1,
    "dp_noise_multiplier": 0.01,
}

TARGET_DIR = GOOGLE_DRIVE_TARGET_DIR

os.makedirs(TARGET_DIR, exist_ok=True)

for experiment_idx in range(N_EXPERIMENTS):
    for option, option_range in options_grid.items():
        options[option] = np.random.uniform(*option_range)

    stringified_options = stringify_options(options)
    serialized_params = "_".join(map(str, options.values()))
    root_output_dir = "{}/{}".format(
        TARGET_DIR, serialized_params
    )
    os.makedirs(root_output_dir, exist_ok=True)
    root_output_dir = root_output_dir.replace(" ", "\ ")
    stringified_options.append("--root_output_dir {}".format(root_output_dir))
    log_file = "{}/log.{}.txt".format(
        root_output_dir,
        serialized_params,
    )
    command = "PYTHONPATH={} python3 -m {} {} 2>&1 | tee {}".format(
        PROJECT_DIR,
        filename_to_module(SCRIPT_FILENAME),
        " ".join(stringified_options),
        log_file,
    )
    bash(command, print_stdout=True, print_stderr=True)


Streaming output truncated to the last 5000 lines.
2020-05-27 15:39:31.062263: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:184] Filling up shuffle buffer (this may take a while): 50 of 10000
2020-05-27 15:39:31.691878: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:184] Filling up shuffle buffer (this may take a while): 14 of 10000
2020-05-27 15:39:31.851147: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:233] Shuffle buffer filled.
2020-05-27 15:39:31.860446: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:233] Shuffle buffer filled.
2020-05-27 15:39:37.913048: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:233] Shuffle buffer filled.
2020-05-27 15:39:38.229488: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:233] Shuffle buffer filled.
2020-05-27 15:39:39.683842: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:233] Shuffle buffer filled.
2020-05-27 15:40:05.563531: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:184] Filling up shuffle b